In [ ]:
import urllib
import urllib.parse
#get - method1
# response = urllib.request.urlopen('https://www.baidu.com')
# print(response.read().decode('utf-8')) #对获取的网页进行utf-8解码
#post - method2 httpbin.org
data = bytes(urllib.parse.urlencode({"hello":"world"}),encoding='utf-8')
postresponse = urllib.request.urlopen('http://www.httpbin.org/post',data=data)
print(postresponse.read().decode('utf-8'))

In [2]:
import urllib
# try:
#     getresponse = urllib.request.urlopen('http://www.httpbin.org/get',timeout=0.01)
#     print(getresponse.read().decode('utf-8'))
# except urllib.error.URLError as e:
#     print("time out!")

In [3]:
getresponse = urllib.request.urlopen('http://www.httpbin.org/get')
print(getresponse.status)

200


In [4]:
# response = urllib.request.urlopen("http://douban.com")
# print(reponse.status)
#error 418 证明对方知道你是一个查重

In [5]:
response = urllib.request.urlopen("http://www.baidu.com")
print(response.getheader("Server"))

BWS/1.1


In [8]:
url2 = "http://httpbin.org/post"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75"
}
data = bytes(urllib.parse.urlencode({"name":"Eric"}),encoding="utf-8")
req = urllib.request.Request(url=url,data=data,headers=headers,method ="POST")
res = urllib.request.urlopen(req)
print(res.read().decode("utf-8"))

<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8" />
<title></title>
<link rel="stylesheet" href="//dragonstatic.com/parking/css/native.css" type="text/css" media="screen" />
<script type="059d933767d5ef4a81979b6e-text/javascript" src="//dragonstatic.com/parking/js/track.js?v=190929"></script>
<script type="059d933767d5ef4a81979b6e-text/javascript">var pageinfo = {ad:1};</script>
<script type="059d933767d5ef4a81979b6e-text/javascript" src="//dragonstatic.com/parking/js/native.js?v=201206"></script>

</head>
<body>
<div id='root'></div>
<script src="https://ajax.cloudflare.com/cdn-cgi/scripts/7089c43e/cloudflare-static/rocket-loader.min.js" data-cf-settings="059d933767d5ef4a81979b6e-|49" defer=""></script></body>
</html>



In [9]:
url = "https://www.doubai.com"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75"
}
req = urllib.request.Request(url=url,headers=headers)
res = urllib.request.urlopen(req)
print(res.read().decode("utf-8"))

<!DOCTYPE html><html data-adblockkey="MFwwDQYJKoZIhvcNAQEBBQADSwAwSAJBANDrp2lz7AOmADaN8tA50LsWcjLFyQFcb/P2Txc58oYOeILb3vBw7J6f4pamkAQVSQuqYsKx3YzdUHCvbVZvFUsCAwEAAQ==_m03SKooon2mopHAMPek9H6B2+rng2YoCq9vX8nWjgoLHN7+dVEM56JL6xHmifeporw4PG49tmDn6JM1QYwbU0Q=="><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><title></title><meta name="viewport" content="width=device-width, initial-scale=1"><meta name="description" content="See related links to what you are looking for."/></head><!--[if IE 6 ]><body class="ie6"><![endif]--><!--[if IE 7 ]><body class="ie7"><![endif]--><!--[if IE 8 ]><body class="ie8"><![endif]--><!--[if IE 9 ]><body class="ie9"><![endif]--><!--[if (gt IE 9)|!(IE)]> --><body><!--<![endif]--><script type="text/javascript">g_pb=(function(){var
DT=document,azx=location,DD=DT.createElement('script'),aAC=false,LU;DD.defer=true;DD.async=true;DD.src="//www.google.com/adsense/domains/caf.js";DD.onerror=function(){if(azx.search!=='?z'){azx.href='/?z';}};DD.onlo

In [2]:
import urllib
def main():
    baseurl = "https://movie.douban.com/top250?start="
    #爬取网页
    datalist = getData(baseurl)
    savepath = "D:\\DownloadFiles\\doubanTop250.xls"
    #保存数据
    saveData(savepath)
    askUrl("https://movie.douban.com/top250?start=")
#爬取网页
def getData(baseurl):
    datalist = []
    #逐一解析数据
    return datalist

#得到指定网页的一个内容
def askUrl(url):
    #让对方服务器认为自己是个浏览器
    header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75"
    }
    request = urllib.request.Request(url,headers=header);
    html = ""
    try:
        response = urllib.request.urlopen(request)
        html = response.read().decode("utf-8")
        print(html)
    except urllib.error.URLError as e:
        if hasattr(e,"code"):
            print(e.code)
        if hasattr(e,"reason"):
            print(e.reason)


#保存数据
def saveData(savePath):
    print("save...")

if __name__ == "__main__":
    main()

save...


NameError: name 'urllib' is not defined